# ⚙️ Initiez-vous au MLOps (partie 1/2)

## 🤖 Modélisation
### 🛠️ Préparez l'environnement de travail
#### 📦 Import des modules python

In [38]:
import os
import shutil
import sys
import mlflow
import warnings
import logging

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
os.unsetenv('MLFLOW_TRACKING_URI')
os.unsetenv('MLFLOW_HOME')

# Réduire le niveau de verbosité de MLflow
logging.getLogger('mlflow').setLevel(logging.ERROR)

import pyarrow
import pandas as pd
import numpy as np
from src.visualization.visu_text import print_title, print_end, print_col, quick_df_info
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report


from src.models.modelization import (
    print_report,
    print_cross_validation_scores,
)

### Préparation du jeu de données

In [39]:
df = pd.read_parquet("../data/processed/survey_lung_cancer_features.parquet", engine='pyarrow')
# Conversion des colonnes booléennes en int pour compatibilité avec certains modèles
bool_cols = df.select_dtypes(include=['bool', 'boolean']).columns
df[bool_cols] = df[bool_cols].astype(int)
# Conversion des colonnes catégorielles en codes numériques
cat_cols = df.select_dtypes(include=['category']).columns
df[cat_cols] = df[cat_cols].apply(lambda x: x.cat.codes)

quick_df_info(df)


┌-------------------------------* Information *-------------------------------┐
├─------- Shape: (17940, 31) - Colonnes:
├─ID                        int64     
├─GENDER                    int64     
├─AGE                       float64   
├─LUNG_CANCER               int64     
├─SMOKING                   int64     
├─YELLOW_FINGERS            int64     
├─ANXIETY                   int64     
├─PEER_PRESSURE             int64     
├─CHRONIC DISEASE           int64     
├─FATIGUE                   int64     
├─ALLERGY                   int64     
├─WHEEZING                  int64     
├─ALCOHOL CONSUMING         int64     
├─COUGHING                  int64     
├─SHORTNESS OF BREATH       int64     
├─SWALLOWING DIFFICULTY     int64     
├─CHEST PAIN                int64     
├─SMOKING_x_AGE             float64   
├─SMOKING_x_ALCOHOL         int64     
├─RESPIRATORY_SYMPTOMS      int64     
├─TOTAL_SYMPTOMS            int64     
├─BEHAVIORAL_RISK_SCORE     int64     
├─SEVERE_SYMPTOMS   

### ⛓️‍💥 Separation du jeu de données

In [40]:
# ================================
# SÉPARATION TRAIN / VALIDATION / TEST
# ================================

y = df["LUNG_CANCER"]
X = df.drop(columns=["LUNG_CANCER"])

print_title(f"Données originales: {X.shape[0]} échantillons")
print_col(f"Distribution: Classe 0: {(y == 0).sum()}, Classe 1: {(y == 1).sum()}")
print_col(f"Proportion classe 1: {(y == 1).mean()*100:.1f}%")
print_end()

# Étape 1: Séparer train+val (80%) / test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X,
    y,
    test_size=0.2,  # 20% pour test
    random_state=42,
    stratify=y,  # Garde la même proportion de classes
)

# Étape 2: Séparer train (60%) / validation (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.25,  # 25% de 80% = 20% du total
    random_state=42,
    stratify=y_temp,
)

# ================================
# VÉRIFICATION DES PROPORTIONS
# ================================

total = len(X)
print_title("RÉPARTITION FINALE:")
print_col(f" Train:      {len(X_train)} échantillons ({len(X_train)/total*100:.1f}%)")
print_col(f" Validation: {len(X_val)} échantillons ({len(X_val)/total*100:.1f}%)")
print_col(f"Test:       {len(X_test)} échantillons ({len(X_test)/total*100:.1f}%)")
print_end()

print_title("VÉRIFICATION STRATIFICATION:")
print_col(f" Original    - Classe 1: {(y == 1).mean()*100:.1f}%")
print_col(f" Train       - Classe 1: {(y_train == 1).mean()*100:.1f}%")
print_col(f" Validation  - Classe 1: {(y_val == 1).mean()*100:.1f}%")
print_col(f" Test        - Classe 1: {(y_test == 1).mean()*100:.1f}%")
print_end()

quick_df_info(X_train)


┌------------------* Données originales: 17940 échantillons *------------------┐
├─Distribution: Classe 0: 2319, Classe 1: 15621
├─Proportion classe 1: 87.1%
└------------------------------------------------------------------------------┘

┌---------------------------* RÉPARTITION FINALE: *---------------------------┐
├─ Train:      10764 échantillons (60.0%)
├─ Validation: 3588 échantillons (20.0%)
├─Test:       3588 échantillons (20.0%)
└------------------------------------------------------------------------------┘

┌-----------------------* VÉRIFICATION STRATIFICATION: *-----------------------┐
├─ Original    - Classe 1: 87.1%
├─ Train       - Classe 1: 87.1%
├─ Validation  - Classe 1: 87.1%
├─ Test        - Classe 1: 87.1%
└------------------------------------------------------------------------------┘

┌-------------------------------* Information *-------------------------------┐
├─------- Shape: (10764, 30) - Colonnes:
├─ID                        int64     
├─GENDER           

### 🤖 Baseline
#### ➡️ Choix des modèles :
- DummyClassifier
- LogisticRegression
- RandomForestClassifier
- XGBClassifier
- LGBMClassifier


In [41]:
dummy_pipeline = ImbPipeline([ 
    ('oversampling', SMOTE(random_state=42)),
    ('scaling', StandardScaler()),
    ('model', DummyClassifier(random_state=42)) 
])
rl_pipeline = ImbPipeline([ 
    ('oversampling', SMOTE(random_state=42)),
    ('scaling', StandardScaler()),
    ('model', LogisticRegression(random_state=42)) 
])
rf_pipeline = ImbPipeline([ 
    ('oversampling', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42)) 
])
xgb_pipeline = ImbPipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', xgb.XGBClassifier(random_state=42))
])
lgb_pipeline = ImbPipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', lgb.LGBMClassifier(
        random_state=42,
        verbose=-1  # Désactiver les warnings
    ))
])

models_pipelines = {
    "Dummy": dummy_pipeline,
    "Logistic Regression": rl_pipeline,
    "Random Forest": rf_pipeline,
    "XGBoost": xgb_pipeline,
    "LightGBM": lgb_pipeline
}


In [42]:
mlflow.set_tracking_uri('http://127.0.0.1:5010')
mlflow.xgboost.autolog(disable=True)
mlflow.lightgbm.autolog(disable=True)
mlflow.autolog(disable=True) # Désactivation de l'autologging pour éviter les conflits

In [ ]:
# Définir l'expérience (ou la créer si elle n'existe pas)
EXPERIMENT_NAME = "lung-cancer-detection_baseline"
try:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
except mlflow.exceptions.MlflowException:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id


model_scores = []
for name, pipeline in models_pipelines.items():
    
    with mlflow.start_run(experiment_id=experiment_id, run_name=f"Run {name}") as parent_run:
        
        # Loguer les paramètres globaux (une seule fois)
        mlflow.log_param("data_version", "v1.0")
        mlflow.log_param("train_samples", len(X_train))
        mlflow.log_param("test_samples", len(X_test))
        mlflow.log_param("features", X_train.shape[1])
        mlflow.log_param("model_type", "baseline_comparison")
        # Plusieurs tags à la fois
        mlflow.set_tags({
            "version": "v1.0",
            "dataset": "survey_lung_cancer_features.parquet",
        })

        mlflow.set_tag("mlflow.note.content", "Modèles de base pour comparaison initiale")
        mlflow.set_tag("mlflow.user", "francois.hellebuyck")
        mlflow.set_tag("mlflow.source.name", "03-Modeling.ipynb")
        mlflow.set_tag("mlflow.source.type", "notebook")
        dataset = mlflow.data.from_pandas(
            df, source='data/processed/survey_lung_cancer_features.parquet', name="survey_lung_cancer_features", targets="LUNG_CANCER"
        )
        mlflow.log_input(dataset, context="training")
        print_title(f"Entraînement du modèle: {name}")
        model = pipeline.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        # Afficher le rapport
        accuracy, precision, recall, f1, f2 = print_report(
            y_test, y_pred, target_names=["Non", "Oui"]
        )

        # 1. Log des paramètres spécifiques au modèle (e.g., type de modèle)
        mlflow.log_param("model_name", name)
        
        # 2. Log des métriques (ce sont des nombres)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("f2_score", f2)

        # 3. Log du modèle (C'est cette étape qui crée l'artefact lourd)
        # MLflow SAUVEGARDE L'ARTEFACT VIA LE SERVEUR MLFLOW DISTANT.
        # if "XGB" in name:
        #     # Utilise la saveur MLflow pour XGBoost
        #     mlflow.xgboost.log_model(
        #         xgb_model=model, 
        #         artifact_path="model"
        #     )
        # elif "GBM" in name:
        #     # Utilise la saveur MLflow pour LightGBM
        #     mlflow.lightgbm.log_model(
        #         lgb_model=model, 
        #         artifact_path="model"
        #     )
        # else:
            # Utilise la saveur MLflow pour scikit-learn (pour les autres modèles)
        mlflow.sklearn.log_model(
            sk_model=pipeline, 
            artifact_path="model"
        )
    

        model_score = {
            "Model": name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-score": f1,
            "F2-score": f2,
        }
        model_scores.append(model_score)
        print_end()

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌----------------------* Entraînement du modèle: Dummy *----------------------┐

┌--------------------------* PERFORMANCE DU MODÈLE *--------------------------┐
├─ Accuracy (Exactitude):       0.129 │ (TP+TN)/(TP+TN+FP+FN)
├─ Precision (Précision):       0.000 │ TP/(TP+FP) | Minimiser les faux positifs. 
├─ Recall (Sensibilité):        0.000 │ TP/(TP+FN) | Minimiser les faux négatif. 
├─ F1-score:                    0.000 │ 2*Precision*Recall/(Precision+Recall)
├─ F2-score:                    0.000 │ 5*Precision*Recall/(4*Precision+Recall) | Privilégie le rappel)
└------------------------------------------------------------------------------┘

┌---------------------------* MATRICE DE CONFUSION *---------------------------┐
├─ RÉALITÉ \ PRÉDICTION
|                      Non         Oui
|          Non         464           0
|          Oui        3124           0
|
├─ Détail (classe positive = Oui):
├─ ✅ Vrais Positifs (TP):  0
├─ ❌ Faux Positifs (FP):   0
├─ ❌ Faux Négatifs (FN):   312

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌--------------------------* PERFORMANCE DU MODÈLE *--------------------------┐
├─ Accuracy (Exactitude):       0.869 │ (TP+TN)/(TP+TN+FP+FN)
├─ Precision (Précision):       0.871 │ TP/(TP+FP) | Minimiser les faux positifs. 
├─ Recall (Sensibilité):        0.997 │ TP/(TP+FN) | Minimiser les faux négatif. 
├─ F1-score:                    0.930 │ 2*Precision*Recall/(Precision+Recall)
├─ F2-score:                    0.969 │ 5*Precision*Recall/(4*Precision+Recall) | Privilégie le rappel)
└------------------------------------------------------------------------------┘

┌---------------------------* MATRICE DE CONFUSION *---------------------------┐
├─ RÉALITÉ \ PRÉDICTION
|                      Non         Oui
|          Non           1         463
|          Oui           8        3116
|
├─ Détail (classe positive = Oui):
├─ ✅ Vrais Positifs (TP):  3116
├─ ❌ Faux Positifs (FP):   463
├─ ❌ Faux Négatifs (FN):   8
├─ ✅ Vrais Négatifs (TN):  1
└-----------------------------------------------

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌------------------* Entraînement du modèle: Random Forest *------------------┐

┌--------------------------* PERFORMANCE DU MODÈLE *--------------------------┐
├─ Accuracy (Exactitude):       0.859 │ (TP+TN)/(TP+TN+FP+FN)
├─ Precision (Précision):       0.871 │ TP/(TP+FP) | Minimiser les faux positifs. 
├─ Recall (Sensibilité):        0.984 │ TP/(TP+FN) | Minimiser les faux négatif. 
├─ F1-score:                    0.924 │ 2*Precision*Recall/(Precision+Recall)
├─ F2-score:                    0.959 │ 5*Precision*Recall/(4*Precision+Recall) | Privilégie le rappel)
└------------------------------------------------------------------------------┘

┌---------------------------* MATRICE DE CONFUSION *---------------------------┐
├─ RÉALITÉ \ PRÉDICTION
|                      Non         Oui
|          Non           8         456
|          Oui          50        3074
|
├─ Détail (classe positive = Oui):
├─ ✅ Vrais Positifs (TP):  3074
├─ ❌ Faux Positifs (FP):   456
├─ ❌ Faux Négatifs (FN): 

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌---------------------* Entraînement du modèle: XGBoost *---------------------┐

┌--------------------------* PERFORMANCE DU MODÈLE *--------------------------┐
├─ Accuracy (Exactitude):       0.855 │ (TP+TN)/(TP+TN+FP+FN)
├─ Precision (Précision):       0.870 │ TP/(TP+FP) | Minimiser les faux positifs. 
├─ Recall (Sensibilité):        0.980 │ TP/(TP+FN) | Minimiser les faux négatif. 
├─ F1-score:                    0.922 │ 2*Precision*Recall/(Precision+Recall)
├─ F2-score:                    0.956 │ 5*Precision*Recall/(4*Precision+Recall) | Privilégie le rappel)
└------------------------------------------------------------------------------┘

┌---------------------------* MATRICE DE CONFUSION *---------------------------┐
├─ RÉALITÉ \ PRÉDICTION
|                      Non         Oui
|          Non           6         458
|          Oui          62        3062
|
├─ Détail (classe positive = Oui):
├─ ✅ Vrais Positifs (TP):  3062
├─ ❌ Faux Positifs (FP):   458
├─ ❌ Faux Négatifs (FN): 

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌---------------------* Entraînement du modèle: LightGBM *---------------------┐

┌--------------------------* PERFORMANCE DU MODÈLE *--------------------------┐
├─ Accuracy (Exactitude):       0.869 │ (TP+TN)/(TP+TN+FP+FN)
├─ Precision (Précision):       0.871 │ TP/(TP+FP) | Minimiser les faux positifs. 
├─ Recall (Sensibilité):        0.997 │ TP/(TP+FN) | Minimiser les faux négatif. 
├─ F1-score:                    0.930 │ 2*Precision*Recall/(Precision+Recall)
├─ F2-score:                    0.969 │ 5*Precision*Recall/(4*Precision+Recall) | Privilégie le rappel)
└------------------------------------------------------------------------------┘

┌---------------------------* MATRICE DE CONFUSION *---------------------------┐
├─ RÉALITÉ \ PRÉDICTION
|                      Non         Oui
|          Non           2         462
|          Oui           9        3115
|
├─ Détail (classe positive = Oui):
├─ ✅ Vrais Positifs (TP):  3115
├─ ❌ Faux Positifs (FP):   462
├─ ❌ Faux Négatifs (FN):

#### 📊 Tableau des résultats Baseline

In [44]:
# Tableau des performances des modèles
scores_df = pd.DataFrame(
   model_scores
)
scores_df.set_index("Model", inplace=True)
scores_df.sort_values(by=["F2-score", "Recall"], ascending=False, inplace=True)
print_title("TABLEAU DES PERFORMANCES DES MODÈLES PAR F2-score")
print(scores_df.round(3).to_string(float_format="{:.3f}".format))
print_end()


┌------------* TABLEAU DES PERFORMANCES DES MODÈLES PAR F2-score *------------┐
                     Accuracy  Precision  Recall  F1-score  F2-score
Model                                                               
Logistic Regression     0.869      0.871   0.997     0.930     0.969
LightGBM                0.869      0.871   0.997     0.930     0.969
Random Forest           0.859      0.871   0.984     0.924     0.959
XGBoost                 0.855      0.870   0.980     0.922     0.956
Dummy                   0.129      0.000   0.000     0.000     0.000
└------------------------------------------------------------------------------┘


### ✖️ Validation croisée

In [45]:
models_pipelines = {
    "Logistic Regression": rl_pipeline,
    "Random Forest": rf_pipeline,
    "XGBoost": xgb_pipeline,
    "LightGBM": lgb_pipeline
}
model_scores = []

# Définir l'expérience (ou la créer si elle n'existe pas)
EXPERIMENT_NAME = "lung-cancer-detection_cross-validation"
try:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
except mlflow.exceptions.MlflowException:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

        
for name, pipeline in models_pipelines.items():

    with mlflow.start_run(experiment_id=experiment_id, run_name=f"Run CV {name}") as parent_run:
                
        # Loguer les paramètres globaux (une seule fois)
        mlflow.log_param("data_version", "v1.0")
        mlflow.log_param("train_samples", len(X_train))
        mlflow.log_param("test_samples", len(X_test))
        mlflow.log_param("features", X_train.shape[1])
        mlflow.log_param("model_type", "cross_validation")
        # Plusieurs tags à la fois
        mlflow.set_tags({
            "version": "v1.0",
            "dataset": "survey_lung_cancer_features.parquet",
        })

        mlflow.set_tag("mlflow.note.content", "Modèles avec validation croisée pour évaluation de la stabilité")
        mlflow.set_tag("mlflow.user", "francois.hellebuyck")
        mlflow.set_tag("mlflow.source.name", "03-Modeling.ipynb")
        mlflow.set_tag("mlflow.source.type", "notebook")
        dataset = mlflow.data.from_pandas(
            df, source='data/processed/survey_lung_cancer_features.parquet', name="survey_lung_cancer_features", targets="LUNG_CANCER"
        )
        mlflow.log_input(dataset, context="training CV")

        accuracy_score, precision_score, recall_score, f1_score, f2_score, stable = (
            print_cross_validation_scores(pipeline, X_train, y_train)
        )
         # 1. Log des paramètres spécifiques au modèle (e.g., type de modèle)
        mlflow.log_param("model_name", name)
        
        # 2. Log des métriques (ce sont des nombres)
        mlflow.log_metric("accuracy", accuracy_score)
        mlflow.log_metric("precision", precision_score)
        mlflow.log_metric("recall", recall_score)
        mlflow.log_metric("f1_score", f1_score)
        mlflow.log_metric("f2_score", f2_score)
        mlflow.log_param("stability", stable)

        mlflow.sklearn.log_model(
            sk_model=pipeline, 
            artifact_path="model"
        )
    
        model_score = {
                    "Model": name,
                    "Accuracy": accuracy_score,
                    "Precision": precision_score,
                    "Recall": recall_score,
                    "F1-score": f1_score,
                    "F2-score": f2_score,
                    "Stabilité": stable,
                }
        model_scores.append(model_score)

/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌-----------------------* VALIDATION CROISÉE Pipeline *-----------------------┐
├─ Accuracy: 0.870 (+/- 0.002) [CV: 0.001]
├─ Précision: 0.871 (+/- 0.001) [CV: 0.000]
├─ Rappel:    0.999 (+/- 0.002) [CV: 0.001]
├─ F1-Score:  0.930 (+/- 0.001) [CV: 0.001]
├─ F2-Score:  0.970 (+/- 0.001) [CV: 0.001]
|
├─ Stabilité precision:  0.000 ✅ Stable
├─ Stabilité recall:  0.001 ✅ Stable
├─ Stabilité f1:  0.001 ✅ Stable
├─ Stabilité f2:  0.001 ✅ Stable
└------------------------------------------------------------------------------┘
🏃 View run Run CV Logistic Regression at: http://127.0.0.1:5010/#/experiments/12/runs/4da23db19da74d108ef4a7e18f3fbbf1
🧪 View experiment at: http://127.0.0.1:5010/#/experiments/12


/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌-----------------------* VALIDATION CROISÉE Pipeline *-----------------------┐
├─ Accuracy: 0.854 (+/- 0.007) [CV: 0.004]
├─ Précision: 0.870 (+/- 0.001) [CV: 0.001]
├─ Rappel:    0.978 (+/- 0.008) [CV: 0.004]
├─ F1-Score:  0.921 (+/- 0.004) [CV: 0.002]
├─ F2-Score:  0.954 (+/- 0.006) [CV: 0.003]
|
├─ Stabilité precision:  0.001 ✅ Stable
├─ Stabilité recall:  0.004 ✅ Stable
├─ Stabilité f1:  0.002 ✅ Stable
├─ Stabilité f2:  0.003 ✅ Stable
└------------------------------------------------------------------------------┘
🏃 View run Run CV Random Forest at: http://127.0.0.1:5010/#/experiments/12/runs/1f4cd5ddca8344188b9c7d1de58a4a9b
🧪 View experiment at: http://127.0.0.1:5010/#/experiments/12


/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌-----------------------* VALIDATION CROISÉE Pipeline *-----------------------┐
├─ Accuracy: 0.852 (+/- 0.006) [CV: 0.003]
├─ Précision: 0.871 (+/- 0.002) [CV: 0.001]
├─ Rappel:    0.975 (+/- 0.006) [CV: 0.003]
├─ F1-Score:  0.920 (+/- 0.003) [CV: 0.002]
├─ F2-Score:  0.952 (+/- 0.005) [CV: 0.003]
|
├─ Stabilité precision:  0.001 ✅ Stable
├─ Stabilité recall:  0.003 ✅ Stable
├─ Stabilité f1:  0.002 ✅ Stable
├─ Stabilité f2:  0.003 ✅ Stable
└------------------------------------------------------------------------------┘
🏃 View run Run CV XGBoost at: http://127.0.0.1:5010/#/experiments/12/runs/390618ed0dbe444db4d65df8d8d5258c
🧪 View experiment at: http://127.0.0.1:5010/#/experiments/12


/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/processed/survey_lung_cancer_features.parquet'. Exception: 
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/francoishellebuyck/Documents/projects/openclassrooms/project6/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing value


┌-----------------------* VALIDATION CROISÉE Pipeline *-----------------------┐
├─ Accuracy: 0.869 (+/- 0.001) [CV: 0.001]
├─ Précision: 0.871 (+/- 0.001) [CV: 0.000]
├─ Rappel:    0.998 (+/- 0.001) [CV: 0.001]
├─ F1-Score:  0.930 (+/- 0.001) [CV: 0.000]
├─ F2-Score:  0.970 (+/- 0.001) [CV: 0.001]
|
├─ Stabilité precision:  0.000 ✅ Stable
├─ Stabilité recall:  0.001 ✅ Stable
├─ Stabilité f1:  0.000 ✅ Stable
├─ Stabilité f2:  0.001 ✅ Stable
└------------------------------------------------------------------------------┘
🏃 View run Run CV LightGBM at: http://127.0.0.1:5010/#/experiments/12/runs/02ef40217d014d3b9269643ead0c4d0a
🧪 View experiment at: http://127.0.0.1:5010/#/experiments/12


In [46]:
# Tableau des performances des modèles
scores_df = pd.DataFrame(
   model_scores
)
scores_df.set_index("Model", inplace=True)
scores_df.sort_values(by=["F2-score", "Recall"], ascending=False, inplace=True)
print_title("TABLEAU DES PERFORMANCES DES MODÈLES PAR F2-score")
print(scores_df.round(3).to_string(float_format="{:.3f}".format))
print_end()


┌------------* TABLEAU DES PERFORMANCES DES MODÈLES PAR F2-score *------------┐
                     Accuracy  Precision  Recall  F1-score  F2-score                                                                            Stabilité
Model                                                                                                                                                    
Logistic Regression     0.870      0.871   0.999     0.930     0.970  {'precision': '✅ Stable', 'recall': '✅ Stable', 'f1': '✅ Stable', 'f2': '✅ Stable'}
LightGBM                0.869      0.871   0.998     0.930     0.970  {'precision': '✅ Stable', 'recall': '✅ Stable', 'f1': '✅ Stable', 'f2': '✅ Stable'}
Random Forest           0.854      0.870   0.978     0.921     0.954  {'precision': '✅ Stable', 'recall': '✅ Stable', 'f1': '✅ Stable', 'f2': '✅ Stable'}
XGBoost                 0.852      0.871   0.975     0.920     0.952  {'precision': '✅ Stable', 'recall': '✅ Stable', 'f1': '✅ Stable', 'f2': '✅ Stab

In [47]:
scores_df.iloc[0,:]


Accuracy                                              0.869844
Precision                                             0.870722
Recall                                                0.998826
F1-score                                              0.930385
F2-score                                              0.970276
Stabilité    {'precision': '✅ Stable', 'recall': '✅ Stable'...
Name: Logistic Regression, dtype: object